### Agents Vs Chains

* Chain: the pipline is pre-fixed


* Agent: dynamic decision making 
    * Language models' intelligence make decisions on the fly




<img src="documents/Agent.jpeg" style="width:50%;">




### Tavily Search Engine

Tavily’s Search API is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

<h5>Get API Key:</h5>

* Go to https://tavily.com

* Register (there is a free plan without credit card)

* Get the API key

* Add to the .env file in this folder: 
* * TAVILY_API_KEY=your-key

<hr>

In [1]:
## import API keys from .env file in this folder
import os
from dotenv import load_dotenv
load_dotenv(".env")

openai_api_key    = os.getenv("openai_api_key")
TAVILY_API_KEY    = os.getenv("TAVILY_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
search.invoke({"query": "What does CompuFlair company do?"})

### Retriever
We use the previous video's retriever to load a pdf article on cancer
<hr>

In [3]:
## review article about Liver Cancer
!open documents/APJCP-18-863.pdf

## review article about Pancreatic Cancer
!open documents/nihms-1828057.pdf

In [4]:
## pdf loader
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("documents/nihms-1828057.pdf")
pages = loader.load_and_split()

## text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

## embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
# from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## vector database
from langchain_community.vectorstores import FAISS
vector = FAISS.from_documents(documents, embeddings)
retriever_panc = vector.as_retriever()

In [5]:
## do the same for Liver Cancer 
## pdf loader

loader = PyPDFLoader("documents/APJCP-18-863.pdf")
pages = loader.load_and_split()

## text splitter
documents = text_splitter.split_documents(pages)

## vector database
vector = FAISS.from_documents(documents, embeddings)
retriever_liver = vector.as_retriever()

### Turn retriever into a tool 

Once 
<hr>

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool_panc = create_retriever_tool(
    retriever_panc,
    "pancreatic_cancer_search",
    "Search for information about Pancreatic Cancer. For any questions about Pancreatic Cancer, you must use this tool!",
)

In [7]:
## do the same for Liver Cancer 
retriever_tool_liver = create_retriever_tool(
    retriever_liver,
    "liver_cancer_search",
    "Search for information about Liver Cancer. For any questions about Liver Cancer, you must use this tool!",
)

### Tools list

Make a python list of the tools to be used by the agent.

<hr>

In [8]:
## tools
tools = [search, retriever_tool_panc, retriever_tool_liver]

### Add LLM

Add the LLM as the reasoning engine of the Agent

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key)

### Agent's Prompt
We choose the prompt we want to use to guide the agent.
It is a list of system, human, and place holder prompt templates.
<hr>

In [10]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Initialize Agent

In [11]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

### Agent Executor

This will execute the agent

In [12]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
## run agent executor
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [14]:
agent_executor.invoke({"input": "what is pancreatic cancer?"})



> Entering new AgentExecutor chain...

Invoking: `pancreatic_cancer_search` with `{'query': 'Pancreatic Cancer'}`


Box.
Overview of Pancreatic Cancer
General Facts
Approximately 60 000 diagnoses per year in the US
Incidence of about 1% over lifetime
The tenth to eleventh leading cause of cancer in the US
Third leading cause of cancer-related mortality
5-year survival (all-comers), 10%
Median age at diagnosis, 71 years
Male/female incidence ratio: 1.3/1.0
50% of patients present with metastatic disease (AJCC stage IV)
30% of patients present with locally advanced disease (AJCC stage III)
20% of patients present with localized resectable disease (AJCC stage I and II)
Most common causative germline alterations: BRCA2, BRCA1, ATM, PALB2
Common sites of metastasis: liver, lymph node, lung, and peritoneum
Rare sites of metastasis: skin, brain, and leptomeninges
Lifestyle Risk Factors
Tobacco
Excess alcohol consumption (chronic pancreatitis)
Obesity (body mass index >30), metabolic disorde

{'input': 'what is pancreatic cancer?',
 'output': 'Pancreatic cancer is a relatively uncommon cancer that affects approximately 60,000 people in the US each year. It is the tenth to eleventh leading cause of cancer in the US and the third leading cause of cancer-related mortality. The 5-year survival rate for all patients is 10%. Pancreatic cancer is more common in individuals aged 71 years and older, with a slightly higher incidence in males compared to females.\n\nRisk factors for pancreatic cancer include tobacco use, excess alcohol consumption (which can lead to chronic pancreatitis), obesity, and a diet high in fat and processed meats. Genetic factors such as BRCA2, BRCA1, ATM, and PALB2 mutations can also increase the risk of developing pancreatic cancer.\n\nPatients with pancreatic cancer may present with metastatic disease, locally advanced disease, or localized resectable disease. Common sites of metastasis include the liver, lymph nodes, lungs, and peritoneum. Rare sites of 

In [15]:
agent_executor.invoke({"input": "what is liver cancer?"})



> Entering new AgentExecutor chain...

Invoking: `liver_cancer_search` with `{'query': 'Liver Cancer'}`


Asian Pacific Journal of Cancer Prevention, Vol 18863
DOI:10.22034/APJCP .2017.18.4.863
A Review of Hepatocellular Carcinoma 
Asian Pac J Cancer Prev,  18 (4) , 863-872
Introduction
Hepatocellular carcinoma (HCC), the primary cancer 
of the liver, is derived from hepatocytes and occurs in 
more than approximately 80% of cases of liver cancer 
(Jemal, 2011).
The incidence of HCC is higher on mean ages from its 
diagnosis between 55 and 59 years in China, and 63-65 
years in North America and Europe. Higher rates of HCC 
are observed in East and Southeast Asia, East and Western 
Africa (Jemal et al., 2011; El-Serag, 2012). The latest 
data estimated by GLOBOCAN showed about 782,000 
new cases and 745,000 deaths of liver cancer in 2012 
worldwide. Therefore, the World Health Organization 
(WHO) considers HCC as the second leading cause of 
cancer deaths (Ferlay et al., 2015).
HCC de

{'input': 'what is liver cancer?',
 'output': 'Liver cancer, specifically hepatocellular carcinoma (HCC), is the primary cancer of the liver that is derived from hepatocytes. It accounts for more than approximately 80% of cases of liver cancer. The incidence of HCC is higher in certain regions such as East and Southeast Asia, East and Western Africa. Environmental and genetic factors play a role in the development of HCC, with liver cirrhosis, hepatitis B and C virus infection, excessive alcohol consumption, ingestion of aflatoxin B1, and nonalcoholic steatohepatitis (NASH) being important risk factors.\n\nTreatment options for HCC include surgical resection, liver transplant, and local ablation in the early stages to improve patient survival. However, in advanced stages, available therapies are often limited to palliative care and local treatment. Early detection and appropriate therapy are crucial for increasing survival rates and improving the quality of life for patients with HCC. 

In [16]:
agent_executor.invoke({"input": "search the the web to see what is heart disease?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'What is heart disease?'}`


[{'url': 'https://www.cdc.gov/heartdisease/index.htm', 'content': 'Heart Disease. Heart disease is the leading cause of death in the United States The term "heart disease" refers to several types of heart conditions. In the United States, the most common type of heart disease is coronary artery disease (CAD), which can lead to heart attack. You can greatly reduce your risk for heart disease through ...'}, {'url': 'https://my.clevelandclinic.org/health/diseases/24129-heart-disease', 'content': 'Heart disease is a variety of issues that can affect your heart, but coronary artery disease (CAD) is the most common and familiar one. Learn about the symptoms, causes, types and treatment of heart disease from Cleveland Clinic, a non-profit academic medical center.'}, {'url': 'https://www.heart.org/en/health-topics/consumer-healthcare/what-is-cardiovascular-disease', 'con

{'input': 'search the the web to see what is heart disease?',
 'output': 'Heart disease refers to several types of heart conditions that can affect the heart and blood vessels. The most common type of heart disease in the United States is coronary artery disease (CAD), which can lead to a heart attack. Risk factors for heart disease include high blood pressure, high cholesterol, smoking, diabetes, and obesity. Symptoms of heart disease vary depending on the type of heart condition. It is important to reduce the risk of heart disease through healthy lifestyle choices such as maintaining a balanced diet, regular exercise, and avoiding smoking. If you have any concerns about heart disease, it is best to consult with a healthcare provider for personalized advice and treatment options.'}

In [17]:
agent_executor.invoke({"input": "whats the weather in Houston TX?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Houston TX'}`


[{'url': 'https://weather.com/weather/hourbyhour/l/Houston+TX?canonicalCityId=e7763a6187b4cb5fd0f85ad30c23f37f320bfe7e910e6fdbe90b501f206d265c', 'content': "recents\nSpecialty Forecasts\nHourly Weather-Houston, TX\nSunday, November 26\n12 pm\nMostly Cloudy\n1 pm\nMostly Cloudy\n2 pm\nPartly Cloudy\n3 pm\nPartly Cloudy\n4 pm\nPartly Cloudy\n5 pm\nPartly Cloudy\n6 pm\nMostly Cloudy\n7 pm\nMostly Cloudy\n8 pm\nMostly Cloudy\n9 pm\nMostly Cloudy\n10 pm\nPartly Cloudy\n11 pm\nPartly Cloudy\nMonday, November 27\n12 am\nPartly Cloudy\n1 am\nPartly Cloudy\n2 am\nPartly Cloudy\n3 am\nMostly Cloudy\n4 am\nMostly Cloudy\n5 am\nMostly Cloudy\n6 am\nMostly Cloudy\n7 am\nMostly Cloudy\n8 am\nPartly Cloudy\n9 am\nPartly Cloudy\n10 am\nPartly Cloudy\n11 am\nMostly Cloudy\n12 pm\nMostly Cloudy\n1 pm\nMostly Cloudy\n2 pm\nMostly Cloudy\n3 pm\nMostly Cloudy\n4 pm\nCloudy\n5 pm\nClou

{'input': 'whats the weather in Houston TX?',
 'output': 'The weather in Houston, TX is currently mostly cloudy. The temperature is expected to be in the 70s throughout the day with partly cloudy conditions in the evening. If you would like more detailed information, you can visit [this link](https://weather.com/weather/hourbyhour/l/Houston+TX?canonicalCityId=e7763a6187b4cb5fd0f85ad30c23f37f320bfe7e910e6fdbe90b501f206d265c).'}